# RNN QA
* "Learning to Execute"http://arxiv.org/abs/1410.4615
* "Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Two digits inverted:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs
Three digits inverted:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs
Four digits inverted:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs
Five digits inverted:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs

In [2]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


In [3]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    def encode(self, C, num_rows):
        # Arguments
        #num_rows: Number of rows in the returned one hotencoding
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

* 使用随机数的方法分别生成两个数字，将两个数字相加，然后添加到问题集合中，同样生成结果集合
×　需要进行编码，一个数字是十维变量，一次最大输入七维（）
×　结果最大是四维
×　直接使用one－hot，编码方式也比较容易
×　直接调用LSTM,定义输入跟输出shape即可
×　x跟y标签，损失函数直接选择即可，不用自己定义

In [4]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
INVERT = True
# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAxLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAxLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAxLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if INVERT:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

print('Vectorization...')
x = np.zeros((len(questions), MAxLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAxLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAxLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Generating data...
Total addition questions: 50000
Vectorization...
Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_____________________________

In [5]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 20):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if INVERT else q)
        print('T', correct)
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=" ")
        else:
            print(colors.fail + '☒' + colors.close, end=" ")
        print(guess)
        print('---')


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 58s - loss: 1.8843 - acc: 0.3226 - val_loss: 1.7796 - val_acc: 0.3444
Q 924+97 
T 1021
☒ 100 
---
Q 83+850 
T 933 
☒ 100 
---
Q 5+219  
T 224 
☒ 16  
---
Q 24+423 
T 447 
☒ 166 
---
Q 416+76 
T 492 
☒ 107 
---
Q 848+80 
T 928 
☒ 100 
---
Q 564+1  
T 565 
☒ 66  
---
Q 204+65 
T 269 
☒ 106 
---
Q 682+94 
T 776 
☒ 100 
---
Q 879+995
T 1874
☒ 1107
---

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 55s - loss: 1.7255 - acc: 0.3626 - val_loss: 1.6575 - val_acc: 0.3861
Q 106+25 
T 131 
☒ 266 
---
Q 251+90 
T 341 
☒ 201 
---
Q 8+345  
T 353 
☒ 144 
---
Q 910+380
T 1290
☒ 1303
---
Q 308+207
T 515 
☒ 803 
---
Q 962+15 
T 977 
☒ 103 
---
Q 46+895 
T 941 
☒ 903 
---
Q 254+344
T 598 
☒ 503 
---
Q 82+364 
T 446 

45000/45000 [==============================] - 52s - loss: 0.1727 - acc: 0.9668 - val_loss: 0.1872 - val_acc: 0.9549
Q 918+93 
T 1011
☑ 1011
---
Q 793+5  
T 798 
☑ 798 
---
Q 7+437  
T 444 
☑ 444 
---
Q 299+544
T 843 
☑ 843 
---
Q 747+5  
T 752 
☑ 752 
---
Q 403+8  
T 411 
☑ 411 
---
Q 314+48 
T 362 
☑ 362 
---
Q 61+97  
T 158 
☑ 158 
---
Q 601+650
T 1251
☑ 1251
---
Q 555+763
T 1318
☑ 1318
---

--------------------------------------------------
Iteration 16
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 53s - loss: 0.1408 - acc: 0.9712 - val_loss: 0.1398 - val_acc: 0.9666
Q 76+21  
T 97  
☑ 97  
---
Q 165+25 
T 190 
☑ 190 
---
Q 1+217  
T 218 
☑ 218 
---
Q 75+958 
T 1033
☑ 1033
---
Q 47+516 
T 563 
☑ 563 
---
Q 11+142 
T 153 
☑ 153 
---
Q 85+210 
T 295 
☑ 295 
---
Q 836+858
T 1694
☒ 1794
---
Q 49+6   
T 55  
☑ 55  
---
Q 30+650 
T 680 
☑ 680 
---

--------------------------------------------------
Iteration 17
Train on 45000 sa